In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv
import os
import pinecone
import fitz  # PyMuPDF for reading PDF files
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
import os
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
from langchain.schema import Document
import torch
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
import os
import torch
import uuid  # For unique chunk IDs
import pinecone

# Load environment variables
load_dotenv()

# Pinecone API Key and Index Name
api_key = "pcsk_3qCPAf_CX3V2zCFf5ZFVfya8cC1R93wUZQcSPcYvqDGheR8kiLayxw2ZUogQREf4i81y3A"  # Your Pinecone API key
from langchain.vectorstores import Pinecone
import pinecone
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key= api_key)
index = pc.Index("quickstart")

# Initialize Embeddings with GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': device}  # Use GPU if available
)

# Recursive Text Splitter
# Function to read text from PDF"""
"""def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text"""

# Path to your 7 Harry Potter books in PDF format
"""def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents"""
def load_pdfs_from_directory(directory_path):
    loader = DirectoryLoader(directory_path, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents
data_folder = "D:/Animal Classification/data"
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
documents = load_pdfs_from_directory(data_folder)
splitted_documents = text_splitter.split_documents(documents)
vectors = []
for doc in splitted_documents:
    try:
        clean_source = os.path.basename(doc.metadata.get("source", "unknown"))

        # Generate unique ID for each chunk
        chunk_id = str(uuid.uuid4())
        
        # Embed the chunk text
        embedding = embedding_model.embed_query(doc.page_content)
        
        # Add vector with cleaned metadata and the full chunk text
        vectors.append((chunk_id, embedding, {
            "source": clean_source,
            "content": doc.page_content  # Save the actual paragraph for retrieval
        }))
        
        # Batch upsert to Pinecone
        if len(vectors) >= 100:  # Batch size
            index.upsert(vectors=vectors)
            vectors = []  # Clear the vectors list after upsert
    except Exception as e:
        print(f"Error processing document chunk: {e}")

# Upsert any remaining vectors
if vectors:
    index.upsert(vectors=vectors)

print("All PDF files have been processed and indexed successfully.")

Error processing document chunk: Failed to connect; did you specify the correct index name?
Error processing document chunk: HTTPSConnectionPool(host='quickstart-jr20p2p.svc.aped-4627-b74a.pinecone.io', port=443): Max retries exceeded with url: /vectors/upsert (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001A3DAE83C90>: Failed to resolve 'quickstart-jr20p2p.svc.aped-4627-b74a.pinecone.io' ([Errno 11001] getaddrinfo failed)"))
All PDF files have been processed and indexed successfully.


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os
import pinecone
import fitz 
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
import os
import torch
import uuid  # For unique chunk IDs
import pinecone # PyMuPDF for reading PDF files

# Load environment variables
load_dotenv()

# Pinecone API Key and Index Name
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENV = os.getenv("PINECONE_ENV")  # e.g., "us-west1-gcp"
INDEX_NAME = "harry-potter-knowledge"

# Initialize Pinecone
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)
if INDEX_NAME not in pinecone.list_indexes():
    pinecone.create_index(INDEX_NAME, dimension=384)  # Dimension must match the embedding model
index = pinecone.Index(INDEX_NAME)

# Initialize Embeddings
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

# Recursive Text Splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# Function to read text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

# Path to your 7 Harry Potter books in PDF format
pdf_files = [
    "path/to/harry_potter_book1.pdf",
    "path/to/harry_potter_book2.pdf",
    "path/to/harry_potter_book3.pdf",
    "path/to/harry_potter_book4.pdf",
    "path/to/harry_potter_book5.pdf",
    "path/to/harry_potter_book6.pdf",
    "path/to/harry_potter_book7.pdf",
]

# Process each PDF and upload chunks to Pinecone
for pdf_file in pdf_files:
    try:
        # Extract text from PDF
        text = extract_text_from_pdf(pdf_file)
        # Generate metadata
        metadata = {"source": os.path.basename(pdf_file)}
        # Split text into smaller chunks
        splitted_texts = text_splitter.split_documents([{"text": text, "metadata": metadata}])
        
        # Embed and upsert into Pinecone
        vectors = []
        for doc in splitted_texts:
            embedding = embedding_model.embed_query(doc["text"])
            vectors.append((doc["id"], embedding, doc["metadata"]))
        
        # Upsert to Pinecone in batches
        index.upsert(vectors=vectors)
        print(f"Successfully indexed {len(vectors)} chunks from {pdf_file}")
    except Exception as e:
        print(f"Error processing {pdf_file}: {e}")

print("All PDF files have been processed and indexed.")
